In [1]:
import pandas as pd

In [3]:
path = '../data/jumble_data.json'

df = pd.read_json(path_or_buf=path, lines=True)

In [5]:
df.sort_values("value_date")

,jumbled,unjumbled,value_date
125,They weren’t sure if they were going to make i...,HIGHHOPES,2023-10-07
127,VSNAIH,VANISH,2023-10-07
128,HOMSOT,SMOOTH,2023-10-07
129,SAHRH,HARSH,2023-10-07
130,GRUPE,PURGE,2023-10-07
...,...,...,...
106,LAREFT,FALTER,2023-11-05
5,MALYIF,FAMILY,2023-11-05
107,ENNSUE,UNSEEN,2023-11-05
4,GIELSH,SLEIGH,2023-11-05
